# AlaTB example for ff03-star force field 

In [58]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

### Read GMX files
Read topology and coordinates from GMX files

In [59]:
gro = GromacsGroFile('alaTB_ff03-star_tip3p_solv.gro')

In [60]:
top = GromacsTopFile('alaTB_ff03-star_tip3p.top', \
        periodicBoxVectors=gro.getPeriodicBoxVectors(),
        includeDir='.')

### Simulation workflow
Here we will start with a standard simulation workflow, by first energy minimizing, then running short NVT with constraints, followed by a short NPT run and a production run.

In [61]:
system = top.createSystem(nonbondedMethod=PME, \
        nonbondedCutoff=1*nanometer,
        constraints=HBonds)

#### Defining position restraints
We add position restraints in the heavy atoms using the ```CustomExternalForce``` function.

In [62]:
force = CustomExternalForce(\
            "k*((x - x0)^2 + (y - y0)^2 + (z - z0)^2)")
#force.addGlobalParameter("k", 1000.0*kilojoules_per_mole/nanometer**2)
force.addPerParticleParameter("k")
force.addPerParticleParameter("x0")
force.addPerParticleParameter("y0")
force.addPerParticleParameter("z0")
for i, atom_crd in enumerate(gro.positions):
    k = 1000.0*kilojoules_per_mole/nanometer**2
    x0 = atom_crd[0]
    y0 = atom_crd[1]
    z0 = atom_crd[2]
    if (gro.atomNames[i][0] in ('C', 'N', 'O')) \
            and (gro.atomNames[i][0] not in ('OW')):
        force.addParticle(i,[k, x0, y0, z0])
    else:
        force.addParticle(i,[0., x0, y0, z0])
system.addForce(force)

5

Next we generate the integrator with a ```LangevinIntegrator``` object and pass it on to a ```Simulation``` object. 

In [63]:
integrator = LangevinIntegrator(300*kelvin, \
                        1/picosecond, \
                        0.002*picoseconds)

In [64]:
nvt_posre = Simulation(top.topology, system, \
                        integrator)

We use the gro file atomic positions as initial positions for the simulation.

In [65]:
nvt_posre.context.setPositions(gro.positions)

First, we run a quick energy minimization.

In [66]:
nvt_posre.minimizeEnergy()

We define the reporters for both time series of conformations and also for energies.

In [67]:
pdb_rep = PDBReporter('nvt_posre.pdb', 1000)
nvt_posre.reporters.append(pdb_rep)
state_rep = StateDataReporter(stdout, 100, \
                step=True, potentialEnergy=True, \
                temperature=True, volume=True, density=True)
nvt_posre.reporters.append(state_rep)

In [68]:
nvt_posre.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
100,-32564.727455615124,53.46662780472935,20.679574978308164,0.9753715458022438
200,-31748.719643115124,97.5587749863061,20.679574978308164,0.9753715458022438
300,-31093.516518115124,133.75596590642394,20.679574978308164,0.9753715458022438
400,-30536.008705615124,159.24645945076364,20.679574978308164,0.9753715458022438
500,-30070.883705615124,187.03951510755937,20.679574978308164,0.9753715458022438
600,-29675.672768115124,199.81863919402227,20.679574978308164,0.9753715458022438
700,-29334.649330615124,220.01557161247854,20.679574978308164,0.9753715458022438
800,-29085.461830615124,236.1042245433857,20.679574978308164,0.9753715458022438
900,-28775.618080615124,246.18894873191113,20.679574978308164,0.9753715458022438
1000,-28585.493080615124,257.26036809206414,20.679574978308164,0.9753715458022438


#### NPT run
First we remove the position restrains from the system, because we have already equilibrated the water.

In [69]:
nforces = len(system.getForces())
system.removeForce(nforces-1)

In [70]:
state = nvt_posre.context.getState(getPositions=True, \
                    getVelocities=True)


In [71]:
barostat = MonteCarloBarostat(1.0*bar, 300.0*kelvin, 25)

In [72]:
system.addForce(barostat)

5

In [73]:
integrator = LangevinIntegrator(300*kelvin, \
                        10/picosecond, \
                        0.002*picoseconds)

In [74]:
npt = Simulation(top.topology, system, \
                        integrator)
pdb_rep = PDBReporter('npt.pdb', 1000)
nvt_posre.reporters.append(pdb_rep)

npt.context.setState(state)
npt.reporters.append(state_rep)
npt.step(1000)

100,-27794.835019047256,285.15907690592434,20.67153902870408,0.9757507162440984
200,-27301.565547821316,296.53725142228103,20.55314093775467,0.9813716100235793
300,-27295.062997171015,294.98063670429536,20.444035258433864,0.9866089917255894
400,-27107.291680844937,294.03844084792354,20.35821239269016,0.990768178662297
500,-27021.336777037475,294.37536211983576,20.301945314577473,0.993514104219506
600,-26914.303310746007,299.5417562039174,20.256755953902683,0.9957304643955003
700,-26764.788525876706,297.6433916026792,20.47422710937537,0.9851541113310035
800,-26757.90499461838,300.1056227778029,20.603584591944777,0.9789689227675288
900,-26932.02999461838,309.6063102656108,20.603584591944777,0.9789689227675288
1000,-26812.275182014244,300.28889858604714,20.60997275225287,0.9786654866353973


### Production NVT

In [75]:
nforces = len(system.getForces())
system.removeForce(nforces-1)

In [76]:
state = npt.context.getState(getPositions=True, \
                    getVelocities=True)

In [77]:
integrator = LangevinIntegrator(300*kelvin, \
                        10/picosecond, \
                        0.002*picoseconds)

In [78]:

platform = Platform.getPlatformByName('CPU')
properties = {'CpuThreads': '1'}

In [79]:
nvt = Simulation(top.topology, system, \
                integrator, platform, \
                properties)
dcd_rep = DCDReporter('nvt.dcd', 1000)
nvt.reporters.append(dcd_rep)
#pdb_rep = PDBReporter('nvt.pdb', 1000)
#nvt.reporters.append(pdb_rep)
state_rep = StateDataReporter("nvt.txt", 100, \
                step=True, potentialEnergy=True, \
                temperature=True, volume=True, density=True)
nvt.reporters.append(state_rep)

nvt.context.setState(state)
nvt.step(10000)